In [1]:
import json
import numpy as np

## Load data
The documents are kickstarter campaign descriptions. This creates a list of the input documents.

In [2]:
with open("kickstarter.json") as f:
    documents = [json.loads(d)["text"] for d in f.readlines()]
    

print("Loaded {} documents".format(len(documents)))
print("Here is one of them:")
print(documents[0])

Loaded 259227 documents
Here is one of them:
The world of Terra has suffered a Cataclysmic event. The landscapes and cities have changed completely and all organics have mutated.


## Create term-document matrix
Create term-document matrix with tf-scores, here we are using sci-kit learn, but you could also use your code from the last exercise sheet, if you do not want to install this package. 
TfidfVectorizer is a very handy tool, it will do all the preprocessing for you: tokenization, stop word removal, lower casing. You can even set the treshhold for cutting words that occur too few or too many times (parameters max_df and min_df)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
help(TfidfVectorizer)

Help on class TfidfVectorizer in module sklearn.feature_extraction.text:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 |

In [5]:
vectorizer = TfidfVectorizer(stop_words = 'english', max_df = .8, min_df=50)
term_doc_M = vectorizer.fit_transform([doc for doc in documents]).transpose()

In [6]:
print(type(term_doc_M))
print(term_doc_M.shape)

<class 'scipy.sparse.csc.csc_matrix'>
(7169, 259227)


## Decomposition with SVD
[for further reference](https://en.wikipedia.org/wiki/Latent_semantic_analysis)  
$X = U \Sigma V^{T}$   
In the original term-document space the matrix holding the values for term-term similarities is $XX^{T}$, which using the decomposition becomes:  
$XX^{T} = U \Sigma^{2} U^{T}$  
Which means that the new term space is $U \Sigma$  
For document-document similarities:  
$X^{T}X=V \Sigma^{2} V^{T}$  
So the new document space is $V \Sigma $  

In [7]:
from scipy.sparse.linalg import svds
u, s, v_trans = svds(term_doc_M, k = 140)

In [8]:
print(u.shape)
print(s.shape)
print(v_trans.shape)


(7169, 140)
(140,)
(140, 259227)


** Question (1 point)** Plot the singular values in the cell below. Based on the plot  propose a number for the dimensions of the new term-space, and give as the parameter k for the svd. Explain why you chose this value.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


** Question (2 points)** Redo the decomposition with the suggested k, then compute the new document and the new term space. Keep the variable names as suggested below.

In [22]:
words_compressed = 0
docs_compressed = 0

Let's try out the new term matrix. $U \Sigma^{2} U^{T}$ includes all term-term similarities, but it would be too wastefull to compute the whole matrix. The function below implements the dot product of the term matrix with one term, and outputs the ten closest terms.

In [23]:
#row normalize
from sklearn.preprocessing import normalize
words_compressed = normalize(words_compressed, axis = 1)

word_to_index = vectorizer.vocabulary_
index_to_word = {i:t for t,i in word_to_index.iteritems()}
def closest_words(word_in, k = 10):
    if word_in not in word_to_index: return "Not in vocab."
    sims = words_compressed.dot(words_compressed[word_to_index[word_in],:])
    asort = np.argsort(-sims)[:k+1]
    return [(index_to_word[i],sims[i]/sims[asort[0]]) for i in asort[1:]]

In [ ]:
closest_words("record")


** Question (2 points) ** Try out a few words:
1. art
2. record
3. holiday
4. funny
5. easy
6. hard
7. fight
8. pizza
9. children

Give a short qualitative analysis of results that you find interesting. Does it find words that have the same POS-tag? Does it find words that are synonyms (i.e. it could be substituted to the word in question) or rather words that co-occur with the query word. How can you explain this based on the design of the original term-document matrix?

** Question (2 points) **
In the original term-document matrix you had  the documents as features. Can you think of a better/different matrix design?

## Word2Vec-CBOW features
 
Word2Vec-CBOW is another model for learning continous word representations. In the CBOW model the number of dimensions of the words is a hyperparamer, which the designer gets to decide. The CBOW architecture predicts the current word based on the vector representation of the previous N and the future N context words. N is also a hyperparameter. It will learn representations that help this prediction: words with similar distribution will have similar representations.  
Find a detailed description [here](http://arxiv.org/pdf/1411.2738v4.pdf)  
CBOW is an example of the new generation of distributional semantics models. It phrases the learning of the representations as a language modeling task as opposed to the older, count-based models. 
 
  


Let's train a model with context window size 5, and cut the words which have a count less than 5. Size defines the number of dimensions of a word. This may take a while...

In [25]:
from nltk import word_tokenize
tok_docs=[word_tokenize(doc) for doc in documents]

In [26]:
from gensim import models

W2V_model = models.Word2Vec(tok_docs, window=5, min_count=5, workers=4, size=100)

** Question (2 points) ** Try out the same list of words as above. Do you find that this model performs differently in terms of finding synonyms/ words that could be substituted to the word in question?

In [ ]:
W2V_model.most_similar(positive=["record"])

** Question (1 point)** Try out different window sizes (e.g. 1 and 10). Does it change the performance? Try the same with changing the size of the word vectors. Report your findings in a few sentence.

## Something cool to try out

You can not only look for words that are close to your query, but you can also define negative query words: from which you want the result to be far. If you combine these you can have very interesting results.



In [8]:
W2V_model.most_similar(positive=['daughter', 'man'], negative=['woman'], topn=1)

[(u'son', 0.814849853515625)]